In [66]:
import pandas as pd
import numpy as np
import warnings 
import geopandas as gpd
import requests, zipfile, io
from geopy import distance
warnings.filterwarnings("ignore")

In [ ]:
blockgroupshp = "https://www2.census.gov/geo/tiger/TIGER2023/BG/tl_2023_48_bg.zip"
r = requests.get(blockgroupshp)
zf = zipfile.ZipFile(io.BytesIO(r.content))
zf.extractall()
gdf = gpd.read_file('tl_2023_48_bg.shp')

In [14]:
gdf['centroid'] = gdf['geometry'].centroid
gdf['Block Group Centroid Longitude'] = gdf['centroid'].x
gdf['Block Group Centroid Latitude'] = gdf['centroid'].y
block_group_coordinates = gdf[['GEOIDFQ', 'Block Group Centroid Latitude', 'Block Group Centroid Longitude']]
block_group_coordinates = block_group_coordinates.rename(columns={'GEOIDFQ': 'Geo Index'})

In [ ]:
##Decentennial DHC 2020 Rural vs Urban Population
classifurl = 'https://drive.google.com/file/d/1ViQxHHbZi7X-BkvvYBKUeDOuSvznRFVr/view?usp=share-link'
classifurl='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(classifurl.split('/')[-2])
classifdf = pd.read_csv(classifurl)
classifdf = classifdf[2:].reset_index(drop=True)
classifdf = classifdf.astype({'GEO_ID': 'str', 'NAME': 'str', 'P2_002N': 'int64', 'P2_003N': 'int64'})

In [63]:
classifdf['Classification'] = np.where(classifdf['P2_003N'] > classifdf['P2_002N'], "Rural", "Urban")
classifdf = classifdf[['GEO_ID', 'Classification']].rename(columns={'GEO_ID': 'Geo Index'})

In [15]:
##B25002 Occupancy of Housing Units
occurl = 'https://drive.google.com/file/d/1v4YateaDR-UTQm6XhbqLMuzv6x1ZYXmF/view?usp=share_link'
occurl='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(occurl.split('/')[-2])
occupancydf = pd.read_csv(occurl)
occupancydf.columns = occupancydf.iloc[0]
occupancydf = occupancydf[1:].reset_index(drop=True)
occupancydf = occupancydf[['Geography', 'Geographic Area Name', 'Estimate!!Total:!!Occupied']]
occupancydf.columns = ['Geo Index', 'Area Name', 'Occupied Housing Units']
occupancydf = occupancydf.astype({'Geo Index': 'str', 'Area Name': 'str', 'Occupied Housing Units': 'int64'})

In [16]:
occupancydf

,Geo Index,Area Name,Occupied Housing Units
0,1500000US480019501001,Block Group 1; Census Tract 9501; Anderson Cou...,611
1,1500000US480019501002,Block Group 2; Census Tract 9501; Anderson Cou...,839
2,1500000US480019501003,Block Group 3; Census Tract 9501; Anderson Cou...,477
3,1500000US480019504011,Block Group 1; Census Tract 9504.01; Anderson ...,26
4,1500000US480019504021,Block Group 1; Census Tract 9504.02; Anderson ...,0
...,...,...,...
18633,1500000US485079503021,Block Group 1; Census Tract 9503.02; Zavala Co...,576
18634,1500000US485079503022,Block Group 2; Census Tract 9503.02; Zavala Co...,213
18635,1500000US485079503023,Block Group 3; Census Tract 9503.02; Zavala Co...,323
18636,1500000US485079503024,Block Group 4; Census Tract 9503.02; Zavala Co...,237


In [17]:
##P1 Population Data
totalpopurl = 'https://drive.google.com/file/d/16Xwj6AaIjUAB59NhU_rq_tMiykrAwzHw/view?usp=share_link'
totalpopurl='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(totalpopurl.split('/')[-2])
totalpopdf = pd.read_csv(totalpopurl)
totalpopdf.columns = totalpopdf.iloc[0]
totalpopdf = totalpopdf[1:].reset_index(drop=True)
totalpopdf = totalpopdf[['Geography', ' !!Total']]
totalpopdf.columns = ['Geo Index', 'Total Population']
totalpopdf = totalpopdf.astype({'Geo Index': 'str', 'Total Population': 'int64'})

In [18]:
totalpopdf

,Geo Index,Total Population
0,1500000US480019501001,1196
1,1500000US480019501002,1962
2,1500000US480019501003,1483
3,1500000US480019504011,5782
4,1500000US480019504021,7115
...,...,...
18633,1500000US485079503021,1928
18634,1500000US485079503022,842
18635,1500000US485079503023,1140
18636,1500000US485079503024,869


In [ ]:
##B17017 Poverty Status (household)
povurl = 'https://drive.google.com/file/d/1Se0lYsstbDZtrOf2aSsfAzs3hSqBn_kf/view?usp=share_link'
povurl='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(povurl.split('/')[-2])
povdf = pd.read_csv(povurl)
povdf = povdf[['GEO_ID', 'B17017_002E', 'B17017_031E']]
povdf = povdf[1:].reset_index(drop=True)
povdf.columns = ['Geo Index', 'Households Below Poverty Level']
povdf = povdf.astype({'Geo Index': 'str', 'Households Below Poverty Level': 'int64'})

In [20]:
povdf

,Geo Index,Households Below Poverty Level,Households At or Above Poverty Level
0,1500000US480019501001,70,541
1,1500000US480019501002,178,661
2,1500000US480019501003,62,415
3,1500000US480019504011,0,26
4,1500000US480019504021,0,0
...,...,...,...
18633,1500000US485079503021,285,291
18634,1500000US485079503022,44,169
18635,1500000US485079503023,48,275
18636,1500000US485079503024,80,157


In [21]:
#B25044 Tenure by Vehicles Available (household)
vehiclesurl = 'https://drive.google.com/file/d/1SanHZUmBEuISsFvPpr2DeVJkDG3XV3Im/view?usp=share_link'
vehiclesurl='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(vehiclesurl.split('/')[-2])
vehiclesdf = pd.read_csv(vehiclesurl)
vehiclesdf = vehiclesdf[['GEO_ID', 'B25044_003E', 'B25044_010E']]
vehiclesdf = vehiclesdf[1:].reset_index(drop=True)
vehiclesdf.columns = ['Geo Index', 'Owner Occupied Households No Vehicle', 'Renter Occupied Households No Vehicle']
vehiclesdf = vehiclesdf.astype({'Geo Index': 'str', 'Owner Occupied Households No Vehicle': 'int64', 'Renter Occupied Households No Vehicle': 'int64'})

In [22]:
vehiclesdf

,Geo Index,Owner Occupied Households No Vehicle,Renter Occupied Households No Vehicle
0,1500000US480019501001,3,15
1,1500000US480019501002,0,10
2,1500000US480019501003,20,0
3,1500000US480019504011,0,0
4,1500000US480019504021,0,0
...,...,...,...
18633,1500000US485079503021,20,0
18634,1500000US485079503022,0,0
18635,1500000US485079503023,11,0
18636,1500000US485079503024,10,0


In [24]:
#Fast food locations for locating food swamps
ffurl = 'https://drive.google.com/file/d/1i16keSSE16ysh0g2q_0hwgIBrzPI2zZ2/view?usp=share_link'
ffurl='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(ffurl.split('/')[-2])
ffdf = pd.read_csv(ffurl)
ffdf = ffdf[['city', 'province', 'country', 'latitude', 'longitude']]
ffdf = ffdf[(ffdf['country'] == 'US') & (ffdf['province'] == 'TX')]
ffdf = ffdf[['latitude', 'longitude']]

In [25]:
ffdf

,latitude,longitude
253,33.466670,-94.043230
515,30.555157,-97.691696
516,29.520780,-95.076780
1505,31.840440,-106.572890
1508,31.883100,-106.583290
...,...,...
9940,32.575938,-96.671489
9942,32.473080,-100.400862
9965,32.576925,-96.309464
9977,29.802502,-95.487346


In [75]:
#SNAP Retailers
snapurl = 'https://drive.google.com/file/d/1v9l5GSJdfdGWU7by6trpF6GQK9yPfAp8/view?usp=share_link'
snapurl='https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(snapurl.split('/')[-2])
snapdf = pd.read_csv(snapurl, encoding='latin1')
snapdf = snapdf[(snapdf['State'] == 'TX')]
snapdf['Authorization Date'] = pd.to_datetime(snapdf['Authorization Date'])
snapdf = snapdf.replace("", None)
snapdf = snapdf.replace(" ", None)
snapdf['End Date'] = pd.to_datetime(snapdf['End Date'])
snapdf = snapdf[(snapdf['Authorization Date'].dt.year <= 2023) & ((snapdf['End Date'].dt.year >= 2023) | snapdf['End Date'].isnull())]
snapdf = snapdf[~snapdf['Store Name'].str.lower().str.contains('costco wholesale')]
snapdf = snapdf[~(snapdf['Store Name'].str.lower().str.contains("sam's club"))]
snapdf = snapdf[~(snapdf['Store Type'].str.lower().str.contains("military commissary"))]
snapdf = snapdf[snapdf['Store Type'].isin(['Supermarket', 'Large Grocery Store', 'Super Store'])]
snapdf = snapdf[['Store Type', 'Latitude', 'Longitude']]

In [76]:
snapdf

,Store Type,Latitude,Longitude
819403,Supermarket,31.50773,-95.47610
819404,Supermarket,31.73475,-95.61121
819468,Large Grocery Store,31.77138,-95.62876
819469,Large Grocery Store,31.77685,-95.61826
819470,Large Grocery Store,31.74482,-95.65524
...,...,...,...
896854,Super Store,33.08250,-98.58324
896869,Large Grocery Store,26.91913,-99.25381
896930,Supermarket,26.90422,-99.26919
896947,Supermarket,26.89368,-99.26291


In [64]:
mergedf = pd.merge(totalpopdf, occupancydf, on='Geo Index', how='left')
mergedf = pd.merge(mergedf, povdf, on='Geo Index', how='left')
mergedf = pd.merge(mergedf, vehiclesdf, on='Geo Index', how='left')
mergedf = pd.merge(mergedf, block_group_coordinates, on='Geo Index', how='left')
mergedf = pd.merge(mergedf, classifdf, on='Geo Index', how='left')
mergedf['Percent Poverty Level'] = mergedf['Households Below Poverty Level'] / mergedf['Occupied Housing Units']
mergedf['Households with No Vehicle Access'] = mergedf['Owner Occupied Households No Vehicle'] + mergedf['Renter Occupied Households No Vehicle']
mergedf = mergedf.drop(columns=['Owner Occupied Households No Vehicle', 'Renter Occupied Households No Vehicle'])
mergedf['Percent No Vehicle Access'] = mergedf['Households with No Vehicle Access'] / mergedf['Occupied Housing Units']
mergedf

,Geo Index,Total Population,Area Name,Occupied Housing Units,Households Below Poverty Level,Households At or Above Poverty Level,Block Group Centroid Latitude,Block Group Centroid Longitude,Classification,Percent Poverty Level,Households with No Vehicle Access,Percent No Vehicle Access
0,1500000US480019501001,1196,Block Group 1; Census Tract 9501; Anderson Cou...,611,70,541,32.061828,-95.499394,Rural,0.114566,18,0.029460
1,1500000US480019501002,1962,Block Group 2; Census Tract 9501; Anderson Cou...,839,178,661,31.962629,-95.496465,Rural,0.212157,10,0.011919
2,1500000US480019501003,1483,Block Group 3; Census Tract 9501; Anderson Cou...,477,62,415,31.974468,-95.612247,Rural,0.129979,20,0.041929
3,1500000US480019504011,5782,Block Group 1; Census Tract 9504.01; Anderson ...,26,0,26,31.737602,-95.820261,Rural,0.000000,0,0.000000
4,1500000US480019504021,7115,Block Group 1; Census Tract 9504.02; Anderson ...,0,0,0,31.797573,-95.915255,Rural,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
18633,1500000US485079503021,1928,Block Group 1; Census Tract 9503.02; Zavala Co...,576,285,291,28.662395,-99.820920,Urban,0.494792,20,0.034722
18634,1500000US485079503022,842,Block Group 2; Census Tract 9503.02; Zavala Co...,213,44,169,28.691480,-99.828843,Urban,0.206573,0,0.000000
18635,1500000US485079503023,1140,Block Group 3; Census Tract 9503.02; Zavala Co...,323,48,275,28.682627,-99.834051,Urban,0.148607,11,0.034056
18636,1500000US485079503024,869,Block Group 4; Census Tract 9503.02; Zavala Co...,237,80,157,28.683151,-99.826568,Urban,0.337553,10,0.042194


In [78]:
closestgrocery = block_group_coordinates.copy()
closestgrocery['dummy'] = 1
closestgrocery['Blockcoords'] = list(zip(closestgrocery['Block Group Centroid Latitude'], closestgrocery['Block Group Centroid Longitude']))
snapdf['dummy'] = 1
snapdf['Snapcoords'] = list(zip(snapdf['Latitude'], snapdf['Longitude']))
closestgrocery = pd.merge(closestgrocery, snapdf, on='dummy', how='outer')
closestgrocery

,Geo Index,Block Group Centroid Latitude,Block Group Centroid Longitude,dummy,Blockcoords,Store Type,Latitude,Longitude,coords,Snapcoords
0,1500000US482150244031,26.300755,-97.985298,1,"(26.300754919416, -97.98529799508509)",Supermarket,31.50773,-95.47610,"(31.50773, -95.4761)","(31.50773, -95.4761)"
1,1500000US482150244031,26.300755,-97.985298,1,"(26.300754919416, -97.98529799508509)",Supermarket,31.73475,-95.61121,"(31.73475, -95.61121)","(31.73475, -95.61121)"
2,1500000US482150244031,26.300755,-97.985298,1,"(26.300754919416, -97.98529799508509)",Large Grocery Store,31.77138,-95.62876,"(31.77138, -95.62876)","(31.77138, -95.62876)"
3,1500000US482150244031,26.300755,-97.985298,1,"(26.300754919416, -97.98529799508509)",Large Grocery Store,31.77685,-95.61826,"(31.77685, -95.61826)","(31.77685, -95.61826)"
4,1500000US482150244031,26.300755,-97.985298,1,"(26.300754919416, -97.98529799508509)",Large Grocery Store,31.74482,-95.65524,"(31.74482, -95.65524)","(31.74482, -95.65524)"
...,...,...,...,...,...,...,...,...,...,...
57815071,1500000US482012528001,29.865404,-95.000995,1,"(29.865403503456324, -95.00099507011562)",Super Store,33.08250,-98.58324,"(33.0825, -98.58324)","(33.0825, -98.58324)"
57815072,1500000US482012528001,29.865404,-95.000995,1,"(29.865403503456324, -95.00099507011562)",Large Grocery Store,26.91913,-99.25381,"(26.91913, -99.25381)","(26.91913, -99.25381)"
57815073,1500000US482012528001,29.865404,-95.000995,1,"(29.865403503456324, -95.00099507011562)",Supermarket,26.90422,-99.26919,"(26.90422, -99.26919)","(26.90422, -99.26919)"
57815074,1500000US482012528001,29.865404,-95.000995,1,"(29.865403503456324, -95.00099507011562)",Supermarket,26.89368,-99.26291,"(26.89368, -99.26291)","(26.89368, -99.26291)"


In [ ]:
# This took an hour to run
# closestgrocery['distance'] = closestgrocery.apply(lambda x: distance.distance(x.Blockcoords, x.Snapcoords).miles, axis=1)
# closestgrocery